# Equal-Weight S&P 500 Index Fund
The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust.

The goal is to create a Python script to calculated the number of shares of each stock needed to be allocated by an investor for an equal-weighted index fund.

import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math 

Import all stocks in S&P 500 index

In [1]:
import pandas as pd
import xlsxwriter
import requests
import math
import numpy as np

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
491    YUM
492    ZBH
493   ZBRA
494   ZION
495    ZTS

[496 rows x 1 columns]


Import api keys and test the connection

In [3]:
from secrets1 import IEX_CLOUD_API_TOKEN 

In [4]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
print(data)

<Response [200]>


## Parsing Our API Call

After successful testing of connection we the data from exchange that isn't in proper format yet, so we need to parse it first.

In [5]:
data = requests.get(api_url).json()
my_columns = ['Ticker','Stock Price', 'Market Capitalization','Number of shares to buy']
final_df = pd.DataFrame(columns = my_columns)
print(final_df)
print(data['symbol'])

Empty DataFrame
Columns: [Ticker, Stock Price, Market Capitalization, Number of shares to buy]
Index: []
AAPL


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.
This is because HTTP requests are typically one of the slowest components of a script.
IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make from 500 to 5 - huge improvement!. Then we will split our list of stocks into groups of 100 and then make a batch API call for each group.

The code below divide lists into sublists for batch api calling

In [6]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i + n]

In [7]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
  #  print(symbol_string[i])
final_df = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_call = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_call).json()
    for sym in symbol_string.split(','):
        final_df = final_df._append(
            pd.Series(
               [
                   sym,
                    data[sym]['quote']['latestPrice'],
                data[sym]['quote']['marketCap'],
                'N/A'
           ],
            index = my_columns),
            ignore_index = True
            )
final_df


/var/folders/kl/g1hpj41559g7jyb25xkdrwdc0000gn/T/ipykernel_45931/2843460568.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = final_df._append(


,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,107.445,31437039547,N/A
1,AAL,11.680,7633353624,N/A
2,AAP,56.330,3349959802,N/A
3,AAPL,185.300,2881924945600,N/A
4,ABBV,137.830,243344022736,N/A
...,...,...,...,...
491,YUM,125.240,35105801348,N/A
492,ZBH,103.715,21674434441,N/A
493,ZBRA,205.940,10576996848,N/A
494,ZION,32.620,4832613823,N/A


## Calculating the Number of Shares to Buy

In [8]:
port_size = input('enter value of portfolio: ')
try:
    val = float(port_size)
    print(val)
except:
    print('enter an integer')
    port_size = input('enter value of portfolio: ')
    val = float(port_size)

enter value of portfolio:  1000000000


1000000000.0


In [9]:
pos_size = val/(len(final_df.index))
for i in range(0, len(final_df.index)):
    final_df.loc[i,'Number of shares to buy'] = math.floor(pos_size/final_df.loc[i,'Stock Price'])
final_df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,107.445,31437039547,18764
1,AAL,11.680,7633353624,172613
2,AAP,56.330,3349959802,35791
3,AAPL,185.300,2881924945600,10880
4,ABBV,137.830,243344022736,14627
...,...,...,...,...
491,YUM,125.240,35105801348,16098
492,ZBH,103.715,21674434441,19439
493,ZBRA,205.940,10576996848,9789
494,ZION,32.620,4832613823,61806


## Formatting Our Excel Output
Using the XlsxWriter library 
### Initializing our XlsxWriter Object

In [10]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File
We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [11]:
backc = '#0a0a23'
fc = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color':fc,
        'bg_color': backc,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':fc,
        'bg_color': backc,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color':fc,
        'bg_color': backc,
        'border': 1
    }
)

In [12]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving changes

writer.close()